In [1]:
import torch
import os
import matplotlib.pyplot as plt
import numpy as np
from cpgan import init_yaml
from cpgan.eval.models_lib.ex6 import generator
import pandas as pd
from cpgan.ooppnm import img_process
from cpgan.ooppnm import pnm_sim
import random
import porespy as ps
from cpgan.eval.util import z_perturb
import pickle
from cpgan.ooppnm import pnm_sim_boundary
from cpgan.ooppnm import pnm_sim_old

import sys
import io

class SuppressPrint:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = io.StringIO()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout



f_yaml = init_yaml.yaml_f
img_prc = img_process.Image_process()

def load_gan(ex,epoch):
    f_yaml = init_yaml.yaml_f
    gen_path = os.path.join(f_yaml['model'],"ex{}/cganex{}-{}.pth".format(ex,ex,epoch))
    gen = generator.Generator(z_dim=200)
    gen.load_state_dict(torch.load(gen_path,map_location=torch.device('cpu')))
    gen.eval()
    return gen

def z_to_img(z,gen):
    img = img_prc.clean_img_filt( gen(z) )[0]
    return img

#### Control porosity generation

In [2]:
ex = 6
epoch_gan = 15

# load model and features
gen = load_gan(ex,epoch_gan)
phys_func = img_prc.phi

target_phi = 0.2

opt = z_perturb.Z_perturb(gen,phys_func,target_phi)
z,_,_,epoch = opt.optimize(100,0.01,eta=0.5)
print("epoch: ",epoch)
img = z_to_img(z,gen) > 0.5
print("phi: ",img_prc.phi(img), "target phi: ",target_phi)


Epoch 0, error: 0.019000816345214833
Epoch 1, error: 0.05103187561035155
Epoch 2, error: 0.05421190261840819
Epoch 3, error: 0.030283737182617176
Epoch 4, error: 0.02573423385620116
Epoch 5, error: 0.046705055236816395
Epoch 6, error: 0.03999071121215819
Epoch 7, error: 0.059111881256103505
Epoch 8, error: 0.05529146194458007
Epoch 9, error: 0.05112295150756835
Epoch 10, error: 0.05167846679687499
Epoch 11, error: 0.04166297912597655
Epoch 12, error: 0.03694753646850585
Epoch 13, error: 0.010168361663818348
Epoch 14, error: 0.004305648803710926
epoch:  14
phi:  0.19919586181640625 target phi:  0.2


#### Control absolute perm generation

In [13]:
def kabs_sim(img):
    with SuppressPrint():   
        data_pnm = pnm_sim_old.Pnm_sim(im=img)
        data_pnm.network_extract()
        if data_pnm.error == 1:
            print('Error in network extraction')
            return None
        data_pnm.init_physics()
        data_pnm.get_absolute_perm()
        kabs = data_pnm.data_tmp['kabs']
        data_pnm.close_ws()
    return kabs



target_k = 100
ex = 6
epoch_gan = 15

# load model and features
gen = load_gan(ex,epoch_gan)
opt = z_perturb.Z_perturb(gen,kabs_sim,target_k)
z,_,_,epoch = opt.optimize(100,15,eta=0.5)
print("epoch: ",epoch)
img = z_to_img(z,gen) > 0.5
print("k: ",kabs_sim(img), "target k: ",target_k)



Epoch 0, error: 168.01128843209773
Epoch 1, error: 187.5558705194042
Epoch 2, error: 210.38942895991823
Epoch 3, error: 39.826855572736605
Epoch 4, error: 38.24628988993186
Epoch 5, error: 66.80238781233393
Epoch 6, error: 62.25047341997916
Epoch 7, error: 484.6483449992146
Epoch 8, error: 269.24168195718187
Epoch 9, error: 314.45000203494584
Epoch 10, error: 223.48036093730315
Epoch 11, error: 115.18369232806583
Epoch 12, error: 147.5022637205078
Epoch 13, error: 138.75083035613514
Epoch 14, error: 251.61061924860633
Epoch 15, error: 430.5709475899389
Epoch 16, error: 309.09637930237847
Epoch 17, error: 26.13241675587325
Epoch 18, error: 287.23692143945146
Epoch 19, error: 237.36378903364852
Epoch 20, error: 309.87783109072745
Epoch 21, error: 39.44631090338089
Epoch 22, error: 365.4442369500993
Epoch 23, error: -66.64892097231832
Epoch 24, error: -73.88506576856791
Epoch 25, error: 33.52929040048244
Epoch 26, error: 141.79771817862115
Epoch 27, error: 152.69850854797582
Epoch 28, err

#### Control mean pore size generation

In [ ]:
def psd_sim(img):
    data_pnm = pnm_sim.Pnm_sim(im=img)
    psd, _ = data_pnm.network_extract()
    return psd


#### Control mean Throat size generation